In [1]:
from common_func import *
if type_of_script() == 'jupyter': from tqdm.notebook import tqdm
else: from tqdm import tqdm
    
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D, proj3d
#import cmasher as cmr

import numpy as np
import os, glob, atexit, signal, time, warnings, argparse, subprocess

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle

from importlib import reload
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value

In [2]:
mode1 = 'nh'
iout1 = 1026
repo, rurmode, dp = mode2repo(mode1)
nh1 = uri.RamsesSnapshot(repo, iout1, mode=rurmode)
nhs1 = uri.TimeSeries(nh1)
nhs1.read_iout_avail()
nout1 = nhs1.iout_avail['iout']
gals1 = uhmi.HaloMaker.load(nh1, galaxy=True, double_precision=dp)
hals1 = uhmi.HaloMaker.load(nh1, galaxy=False, double_precision=dp)
database1 = f"/home/jeon/MissingSat/database"
LG1 = pklload(f"{database1}/LG")
keys1 = []
for key in LG1.keys():
    if(LG1[key]['isLG']): keys1.append(key)
keys1 = np.array(keys1)

mode2 = 'nh2'
iout2 = 797
repo, rurmode, dp = mode2repo(mode2)
nh2 = uri.RamsesSnapshot(repo, iout2, mode=rurmode)
nhs2 = uri.TimeSeries(nh2)
nhs2.read_iout_avail()
nout2 = nhs2.iout_avail['iout']
gals2 = uhmi.HaloMaker.load(nh2, galaxy=True, double_precision=dp)
hals2 = uhmi.HaloMaker.load(nh2, galaxy=False, double_precision=dp)
database2 = f"/home/jeon/MissingSat/database/nh2"
LG2 = pklload(f"{database2}/LG")
keys2 = []
for key in LG2.keys():
    if(LG2[key]['isLG']): keys2.append(key)
keys2 = np.array(keys2)

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)
[Output 00797] Age (Gyr) : 11.623 / 13.741, z = 0.17158 (a = 0.8536)


# Eye Check

In [3]:
LG1.keys(), LG2.keys()

(dict_keys([1, 2, 3, 4, 7, 8, 9, 10, 14, 17, 23, 24, 25, 26, 29, 35, 36, 37, 40, 46, 47, 52, 63, 64, 75, 84, 107, 108, 121, 129, 207, 11]),
 dict_keys([2, 12, 45, 14, 3, 19, 18, 25, 37, 131, 143, 89, 10]))

In [4]:
LG1[1].keys(), LG2[2].keys()

(dict_keys(['BGG', 'neighbor', 'sats', 'subs', 'real', 'box', 'UDG', 'isLG']),
 dict_keys(['BGG', 'subs', 'sats', 'real', 'Final_sats', 'Final_subs', 'Final_real', 'box', 'UDG', 'isLG']))

In [5]:
t1 = LG1[1]; t2 = LG2[2]

In [6]:
# Check BGG dtype
print(t1['BGG'].dtype.names)
print(t2['BGG'].dtype.names)

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'halo_mvir', 'halo_rvir', 'fcontam', 'dist', 'central', 'main', 'r200', 'm200', 'r200_code', 'm_star_200', 'm_gas_200', 'fcontam_200', 'rp', 'sfr', 'sfr_tot', 'galaxy_nh2', 'halo_nh2', 'matchrate')
('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'h

In [7]:
# Check subs dtype
print(t1['subs'].dtype.names)
print('\n',t1['subs'].dtype.names==t2['subs'].dtype.names)

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'mcontam', 'r10_mem', 'r50_mem', 'r90_mem', 'r10_vir', 'r50_vir', 'r90_vir', 'r10_max', 'r50_max', 'r90_max', 'sub', 'dink', 'mdm', 'mstar', 'mcold', 'mcell', 'Host', 'r200kpc', 'm200', 'r200', 'mdm_vir', 'mstar_vir', 'mcell_vir', 'mcold_vir')

 True


In [8]:
# Check sats dtype
print(t1['sats'].dtype.names)
print('\n',t1['sats'].dtype.names==t2['sats'].dtype.names)

('nparts', 'id', 'timestep', 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 'aexp', 'm', 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 'sigma_bulge', 'm_bulge', 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'halo_id', 'halo_nparts', 'halo_level', 'halo_host', 'halo_hostsub', 'halo_x', 'halo_y', 'halo_z', 'halo_vx', 'halo_vy', 'halo_vz', 'halo_mvir', 'halo_rvir', 'fcontam', 'dist', 'central', 'main', 'Host', 'r50m', 'r90m', 'r50r', 'r90r', 'SFR_mem', 'u_mem', 'g_mem', 'r_mem', 'i_mem', 'z_mem', 'metal_mem', 'ager_mem', 't50_mem', 't90_mem', 'SFR_r50m', 'u_r50m', 'g_r50m', 'r_r50m', 'i_r50m', 'z_r50m', 'metal_r50m', 'ager_r50m', 't50_r50m', 't90_r50m', 'mgas_r50m', 'mcold_r50m', 'mdm_r50m', 'SFR_r90m', 'u_r90m', 'g_r90m', 'r_r90m', 'i_r90m', 'z_r90m', 'metal_r90m', 'ager_r90m', 't50_r90m', 't90_r90m', 'mgas_r90m', 'mcold_r90m', 'mdm_r90m', 'SFR_r50r', 'u_r50r', 'g_r50r', 'r_r50r', 'i_r50r', 'z_r50r', 'metal_r50r', 'ager_r50r', 

In [9]:
# Check real dtype
print(t1['real'].dtype.names)
print('\n',t1['real'].dtype.names==t2['real'].dtype.names)

('hid', 'gid', 'state')

 True


In [10]:
t1['box'], t2['box']

(array([[0.46326325, 0.47044213],
        [0.47227537, 0.4795758 ],
        [0.51605117, 0.52295538]]),
 array([[0.46339653, 0.46953495],
        [0.47168159, 0.47852275],
        [0.51618915, 0.52290185]]))

In [11]:
t2['real']

array([(21222,     -1, 'dink'), (21224,     -1, 'dink'),
       (21236,     -1, 'dink'), (21240,     -1, 'dink'),
       (21242,     -1, 'dink'), (21243,     -1, 'dink'),
       (21245,     -1, 'dink'), (21248,     -1, 'dink'),
       (21250,     -1, 'dink'), (21252,     -1, 'dink'),
       (21261,     -1, 'dink'), (21264,     -1, 'dink'),
       (21265,     -1, 'dink'), (21267,     -1, 'dink'),
       (21268,     -1, 'dink'), (21269,     -1, 'dink'),
       (21271,     -1, 'dink'), (21272,     -1, 'dink'),
       (21276,     -1, 'dink'), (21277,     -1, 'dink'),
       (21279,     -1, 'dink'), (21280,     -1, 'dink'),
       (21282,     -1, 'dink'), (21285,     -1, 'dink'),
       (21286,     -1, 'dink'), (21287,     -1, 'dink'),
       (21288,     -1, 'dink'), (21289,     -1, 'dink'),
       (21290,     -1, 'dink'), (21294,     -1, 'dink'),
       (21295,     -1, 'dink'), (21296,     -1, 'dink'),
       (21298,     -1, 'dink'), (21299,     -1, 'dink'),
       (21302,     -1, 'dink'),

In [12]:
t2['Final_real']

array([(21222,   -1, 'dink'), (21224,   -1, 'dink'), (21234,   -1, 'ban'),
       (21235,   -1, 'ban'), (21236,   -1, 'dink'), (21238,   -1, 'ban'),
       (21240,   -1, 'dink'), (21242,   -1, 'dink'),
       (21243,   -1, 'dink'), (21244,   -1, 'ban'), (21245,   -1, 'dink'),
       (21248,   -1, 'dink'), (21249,   -1, 'ban'), (21250,   -1, 'dink'),
       (21252,   -1, 'dink'), (21256,   -1, 'ban'), (21257,   -1, 'ban'),
       (21261,   -1, 'dink'), (21263,   -1, 'ban'), (21264,   -1, 'dink'),
       (21265,   -1, 'dink'), (21267,   -1, 'dink'),
       (21268,   -1, 'dink'), (21269,   -1, 'dink'), (21270,   -1, 'ban'),
       (21271,   -1, 'dink'), (21272,   -1, 'dink'), (21273,   -1, 'ban'),
       (21274,   -1, 'ban'), (21275,   -1, 'ban'), (21276,   -1, 'dink'),
       (21277,   -1, 'dink'), (21278,   -1, 'ban'), (21279,   -1, 'dink'),
       (21280,   -1, 'dink'), (21282,   -1, 'dink'), (21283,   -1, 'ban'),
       (21284,   -1, 'ban'), (21285,   -1, 'dink'), (21286,   -1, 'dink'

# Merge LG

In [13]:
LG = {}
LG['nh'] = {}
LG['nh2']= {}
jkeys = ['BGG', 'sats', 'subs', 'real', 'box', 'UDG', 'isLG']
for key in LG1.keys():
    LG['nh'][key] = {}
    for jkey in jkeys:
        LG['nh'][key][jkey] = LG1[key][jkey]
for key in LG2.keys():
    LG['nh2'][key] = {}
    for jkey in jkeys:
        LG['nh2'][key][jkey] = LG2[key][jkey]

In [14]:
LG.keys()

dict_keys(['nh', 'nh2'])

# Main progenitor

In [26]:
tree = pklload(f"{database2}/02_main_progenitors.pickle")

In [55]:
# tmp = tree[tree['timestep'] == 797]
# nsub = len( np.unique(tmp['lastid']) )
# nsub

In [58]:
# newtree = np.hstack( (tree[:nsub], tree[15*nsub:]) )
# pklsave(newtree, f"{database2}/02_main_progenitors.pickle", overwrite=True)